In [ ]:
def moustache(s):
    # Принемает series
    # Возращает значения усов нормального распределения
    q75,q25 = np.percentile(s,[75,25])
    iqr = q75 - q25
    return q25 - 1.5*iqr, q75 + 1.5*iqr

def data_cleanup_table(df):
    # Возвращает таблицу для EDA с анализом признаков
    
    # Создание таблицы
    columns = ['Признак', 'Тип_данных', 'Пропущенных_значений', 
           'Выбросов', 'Ошибочных_значений', 'Управление', 'Комментарий']
    table = pd.DataFrame(columns=columns)
    table['Признак'] = df.columns
    df_len = len(df)
    print('Создание таблицы...')
    #
    for sign in df.columns:
        table.loc[table['Признак'] == sign, 'Тип_данных'] = df[sign].dtype
    
    
    # Анализ пропущенных значений
    for sign in tqdm(df.columns, desc='Анализ пропущенных значений\t'):
        table.loc[table['Признак'] == sign, 'Пропущенных_значений'] = df[sign].isna().sum()
        if df[sign].isna().sum() / df_len > 0.2:
            table.loc[table['Признак'] == sign, 'Управление'] = 'Удалить'
            table.loc[table['Признак'] == sign, 'Комментарий'] = 'Пропусков больше 20% '
            
    
    # Анализ выбрасов
    for sign in tqdm(df.columns, desc='Анализ выбросов\t\t\t'):
        if df[sign].dtype == 'object':
            table.loc[table['Признак'] == sign, 'Выбросов'] = -1
        else:
            tmp = moustache(df[sign]) # возращает границы усов (а, б)
            table.loc[table['Признак'] == sign, 'Выбросов'] = df[(df[sign] < tmp[0]) |
                                                                            (df[sign] > tmp[1])][sign].count()      
    
    return table

In [41]:
# ПРИМЕР:    НЕ ЗАПУСКАТЬ!
table = data_cleanup_table(df_train)
table.head()

Создание таблицы...


Анализ выбросов			: 100%|██████████| 291/291 [00:02<00:00, 106.60it/s]


,Признак,Тип_данных,Пропущенных_значений,Выбросов,Ошибочных_значений,Управление,Комментарий
0,id,int64,0,0,NaN,NaN,NaN
1,timestamp,object,0,-1,NaN,NaN,NaN
2,full_sq,int64,0,963,NaN,NaN,NaN
3,life_sq,float64,6383,0,NaN,Удалить,Пропусков больше 20%
4,floor,float64,167,0,NaN,NaN,NaN


---
---

In [4]:
# Приметивна функция преоброзование даты, надо доработать\

# Обработка дат ДОП!
# df['col'] = pd.to_datetime(df['col']) – меняет тип на datetime64[ns]
# df['col'].dt.day – вытаскивает из столбца col только значение дня
# df['col'].dt.month – только месяц
# df['col'].dt.year – только год


def split_datetime(df):
    day_list = ('Понедельник', 'Вторник', 'Среда', 'Четверг', 'Пятница', 'Суббота', 'Воскресенье')
    df_datetime = df['datetime'].apply(lambda x: dt.strptime(x, '%Y-%m-%d %H:%M:%S'))
    
    df['date'] = df_datetime.apply(lambda x: str(x.date()))
    df['year'] = df_datetime.apply(lambda x: str(x.date()).split('-')[0])
    df['month'] = df_datetime.apply(lambda x: str(x.date()).split('-')[1])
    df['day'] = df_datetime.apply(lambda x: str(x.date()).split('-')[2])
    df['week_number'] = df_datetime.apply(lambda x: x.isocalendar()[1])
    df['weekday'] = df_datetime.apply(lambda x: day_list[x.isocalendar()[2] - 1])
    df['time'] = df_datetime.apply(lambda x: str(x.time()))
    df['hour'] = df_datetime.apply(lambda x: str(x.hour))
    df['min'] = df_datetime.apply(lambda x: str(x.minute))
    df['sec'] = df_datetime.apply(lambda x: str(x.second))
    df['sec_of_day'] = df_datetime.apply(lambda x: str(x.hour*3600 + x.minute*60 + x.second))
    
    return df

df = split_datetime(df)
df.head()
nik_name = 'Igarg'
df.to_excel('../datasets/Даты '+nik_name+'.xlsx', index=False)
df

,datetime,date,year,month,day,week_number,weekday,time,hour,min,sec,sec_of_day
0,2021-01-26 17:22:41,2021-01-26,2021,01,26,4,Вторник,17:22:41,17,22,41,62561
1,2013-04-18 14:44:04,2013-04-18,2013,04,18,16,Четверг,14:44:04,14,44,4,53044
2,2017-11-12 21:51:06,2017-11-12,2017,11,12,45,Воскресенье,21:51:06,21,51,6,78666
3,2001-09-14 22:48:09,2001-09-14,2001,09,14,37,Пятница,22:48:09,22,48,9,82089
4,2012-12-03 18:05:30,2012-12-03,2012,12,03,49,Понедельник,18:05:30,18,5,30,65130
...,...,...,...,...,...,...,...,...,...,...,...,...
95,2002-04-06 14:45:02,2002-04-06,2002,04,06,14,Суббота,14:45:02,14,45,2,53102
96,2010-01-17 19:29:42,2010-01-17,2010,01,17,2,Воскресенье,19:29:42,19,29,42,70182
97,2016-09-20 13:10:02,2016-09-20,2016,09,20,38,Вторник,13:10:02,13,10,2,47402
98,2019-11-27 15:05:00,2019-11-27,2019,11,27,48,Среда,15:05:00,15,5,0,54300
